# 🌱 Foundational RAG Techniques

**Tổng quan**

Đoạn code này triển khai một hệ thống **Retrieval-Augmented Generation (RAG)** cơ bản để xử lý và truy vấn tài liệu PDF. Hệ thống mã hóa nội dung tài liệu thành một **vector store**, sau đó có thể được truy vấn để lấy thông tin liên quan.

**Các thành phần chính**

* Xử lý PDF và trích xuất văn bản
* Chia nhỏ văn bản (**text chunking**) để xử lý dễ dàng
* Tạo **vector store** sử dụng **FAISS** và **embeddings** của OpenAI
* Thiết lập **retriever** để truy vấn các tài liệu đã xử lý
* Đánh giá hệ thống RAG

**Chi tiết phương pháp**

* **Tiền xử lý tài liệu (Document Preprocessing)**
    * PDF được tải bằng PyPDFLoader.
    * Văn bản được chia thành các đoạn nhỏ (**chunks**) bằng RecursiveCharacterTextSplitter với kích thước đoạn và độ chồng lấp được chỉ định.
* **Làm sạch văn bản (Text Cleaning)**
    * Một hàm tùy chỉnh `replace_t_with_space` được áp dụng để làm sạch các đoạn văn bản. Điều này có thể giải quyết các vấn đề định dạng cụ thể trong PDF.
* **Tạo Vector Store**
    * **Embeddings** của OpenAI được sử dụng để tạo biểu diễn vector của các đoạn văn bản.
    * Một **vector store** FAISS được tạo từ các **embeddings** này để tìm kiếm độ tương tự hiệu quả.
* **Thiết lập Retriever**
    * Một **retriever** được cấu hình để lấy 2 đoạn văn bản có liên quan nhất cho một truy vấn nhất định.
* **Hàm mã hóa (Encoding Function)**
    * Hàm `encode_pdf` bao gồm toàn bộ quá trình tải, chia đoạn, làm sạch và mã hóa PDF thành **vector store**.

**Các tính năng chính**

* **Thiết kế mô-đun (Modular Design)**: Quá trình mã hóa được đóng gói trong một hàm duy nhất để dễ dàng tái sử dụng.
* **Chia đoạn có thể cấu hình (Configurable Chunking)**: Cho phép điều chỉnh kích thước đoạn và độ chồng lấp.
* **Truy xuất hiệu quả (Efficient Retrieval)**: Sử dụng FAISS để tìm kiếm độ tương tự nhanh chóng.
* **Đánh giá (Evaluation)**: Bao gồm một hàm để đánh giá hiệu suất của hệ thống RAG.

**Ví dụ sử dụng**

Đoạn code bao gồm một truy vấn thử nghiệm: "What is the main cause of climate change?". Điều này minh họa cách sử dụng **retriever** để lấy ngữ cảnh liên quan từ tài liệu đã xử lý.

**Đánh giá**

Hệ thống bao gồm một hàm `evaluate_rag` để đánh giá hiệu suất của **retriever**, mặc dù các chỉ số cụ thể được sử dụng không được nêu chi tiết trong đoạn code được cung cấp.

**Lợi ích của phương pháp này**

* **Khả năng mở rộng (Scalability)**: Có thể xử lý các tài liệu lớn bằng cách xử lý chúng theo từng đoạn.
* **Tính linh hoạt (Flexibility)**: Dễ dàng điều chỉnh các tham số như kích thước đoạn và số lượng kết quả được truy xuất.
* **Hiệu quả (Efficiency)**: Sử dụng FAISS để tìm kiếm độ tương tự nhanh chóng trong không gian nhiều chiều.
* **Tích hợp với NLP nâng cao (Integration with Advanced NLP)**: Sử dụng **embeddings** OpenAI để biểu diễn văn bản hiện đại.

**Kết luận**

Hệ thống RAG đơn giản này cung cấp nền tảng vững chắc để xây dựng các hệ thống truy xuất thông tin và trả lời câu hỏi phức tạp hơn. Bằng cách mã hóa nội dung tài liệu thành một **vector store** có thể tìm kiếm, nó cho phép truy xuất hiệu quả thông tin liên quan để đáp ứng các truy vấn. Phương pháp này đặc biệt hữu ích cho các ứng dụng yêu cầu truy cập nhanh vào thông tin cụ thể trong các tài liệu lớn hoặc bộ sưu tập tài liệu.


In [8]:
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import helper_functions as helper
import evaluate_rag as eval

In [ ]:
PDF_FILE_PATH = "data/Understanding_Climate_Change.pdf"
# VNEID = "data/vneid.pdf"

### Encode documents

In [20]:
chunks_vector_store = helper.encode_pdf(PDF_FILE_PATH)

### Create retrieve

In [21]:
chunks_query_retrieve = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

### Test retriever

In [22]:
test_query = "What is the main cause of climate change?"
# test_query = "Cách đăng ký tài khoản mức 1?"
context = helper.retrieve_context_per_question(test_query, chunks_query_retrieve)
helper.show_context(context)

Context 1:
Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate. 
Fossil Fuels 
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today. 
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch,

In [23]:
eval.evaluate_rag(chunks_query_retrieve)

{'questions': ["1. What is the primary greenhouse gas responsible for trapping heat in the Earth's atmosphere and contributing to climate change?",
  '2. How do deforestation and land-use changes impact climate change?',
  '3. What are some potential consequences of rising global temperatures due to climate change?',
  '4. How do scientists measure and track changes in global temperatures over time to study climate change?',
  '5. What are some strategies that individuals and communities can implement to mitigate the effects of climate change and reduce their carbon footprint?'],
 'results': ['{\n    "Relevance": 5,\n    "Completeness": 4,\n    "Conciseness": 3\n}',
  '{\n    "Relevance": 5,\n    "Completeness": 4,\n    "Conciseness": 3\n}',
  '{\n    "Relevance": 5,\n    "Completeness": 4,\n    "Conciseness": 3\n}',
  '{\n    "Relevance": 5,\n    "Completeness": 4,\n    "Conciseness": 3\n}',
  '{\n    "Relevance": 5,\n    "Completeness": 4,\n    "Conciseness": 3\n}'],
 'average_scores